## Anomaly Test

In [16]:
import pandas as pd
import numpy as np
from src import iids_util

In [1]:
import pandas as pd
from base import params

def get_groundtruth(dev):
    # loading groundtruth from Edge Datasets
    src = params.edge_fusion_csv_sources.get(dev)
    gt = pd.read_csv(src, low_memory=False)
    colname = params.non_feature + params.feature_naming.get(dev)
    gt = gt[colname].copy()
    gt = gt.sort_values(params.ts_label).copy()
    return gt
    
fog_memory = pd.read_csv(params.csv_sources.get('linux_memory'), low_memory=False)
fog_memory = fog_memory.drop(columns=['CMD_MEM'])
fog_disk = pd.read_csv(params.csv_sources.get('linux_disks'), low_memory=False)
fog_disk = fog_disk.drop(columns=['CMD_DISK'])
fog_process = pd.read_csv(params.csv_sources.get('linux_process'), low_memory=False)
fog_process = fog_process.drop(columns=['CMD_PROCS'])

for name in params.edge_device_list:
    edge_src = params.csv_sources.get(name)
    edge_label = get_groundtruth(dev=name)
    edge_label = pd.merge(edge_label, fog_disk, on=params.ts_label, how='left')
    edge_label = pd.merge(edge_label, fog_memory, on=params.ts_label, how='left')
    edge_label = pd.merge(edge_label, fog_process, on=params.ts_label, how='left')
    edge_label = edge_label.ffill()
    edge_label = edge_label.bfill()
    colname = params.non_feature + params.feature_naming.get(name)
    edge_label = edge_label.groupby(colname).mean().reset_index()
    edge_label = edge_label.dropna().drop_duplicates().sort_values(params.ts_label).reset_index(drop=True)
    print(name, len(edge_label))
    fname = f"{params.fog_dir}Fog_{name}.csv"
    edge_label.to_csv(fname, sep=',', index=False)
    del edge_src, edge_label, fname

fridge 452028
garage_door 179681
modbus 281037
motion_light 222673
thermostat 389638
weather 593295


In [29]:
output = pd.read_csv('output/Anomaly_ouput.csv')
column = output.columns.tolist()
models = ["detector_1", "detector_2", "detector_3"]
for _ in models:
    output[_] = np.round(output[_], 1)
output['count'] = 1

In [30]:
iids_util.evaluation_metrics(y_true=output['y_true'].tolist(),
                             y_pred=output['y_predict'].tolist())

{'accuracy': 49.737,
 'kappa': -0.01,
 'f1_score': 2.883,
 'matthews_corrcoef': -0.03,
 'precision_score': 38.49,
 'recall_score': 1.497}

In [31]:
output.groupby(column).count().reset_index()

,detector_1,detector_2,detector_3,y_predict,y_true,count
0,0.0,0.5,0.3,0,0,268
1,0.0,0.5,0.3,0,1,369
2,0.0,0.5,0.4,0,0,25
3,0.0,0.5,0.4,0,1,86
4,0.0,0.5,0.5,0,0,23
...,...,...,...,...,...,...
218,0.8,1.0,0.5,1,0,11
219,0.8,1.0,0.5,1,1,9
220,0.8,1.0,0.6,1,0,17
221,0.8,1.0,0.6,1,1,12


In [32]:
output.groupby(['y_true', 'y_predict']).count().reset_index()

,y_true,y_predict,detector_1,detector_2,detector_3,count
0,0,0,34152,34152,34152,34152
1,0,1,831,831,831,831
2,1,0,34208,34208,34208,34208
3,1,1,520,520,520,520


In [28]:
(31409 + 3581) / len(output)

0.5019293942132519